In [2]:
import requests
from bs4 import BeautifulSoup
    
def get_most_recent_articles(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('article', class_='news-item') # Adjust the selector based on the website's structure\n",
        return articles[:10]
    
url = 'https://www.mediapool.bg/news/'
articles = get_most_recent_articles(url)


In [3]:
for article in articles:
    title = article.find('h1')  # Adjust the tag name as per actual structure
    if title:
        print(title.text)
    print("---------")


In [4]:
import requests

def fetch_data():
    # Assuming API endpoint that returns articles and summaries
    response = requests.get("https://api.example.com/news")
    data = response.json()
    articles = [{'title': item['title'], 'link': item['url']} for item in data['articles']]
    summaries = [item['summary'] for item in data['articles']]
    return articles, summaries

# Using the function
if __name__ == "__main__":
    articles, summaries = fetch_data()
    display_articles(articles, summaries)


ConnectionError: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /news (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000147BB04D7F0>: Failed to resolve 'api.example.com' ([Errno 11001] getaddrinfo failed)"))

In [5]:
import streamlit as st

def display_articles(articles, summaries):
    for i, (article, summary) in enumerate(zip(articles, summaries)):
        st.header(f"Article {i+1}")
        st.write(f"Title: {article.title}")
        st.write(f"Link: {article.link}")
        st.write("Summary:")
        st.write(summary)

display_articles(articles, summaries)

NameError: name 'summaries' is not defined